# 1.Divide Region



In [1]:
# import libraries
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
import geopandas
import matplotlib.pylab as pylab
# import geoplot
pylab.rcParams['figure.figsize'] = 20, 20

In [2]:
#file_path_definitions
Age_limits_CSV_FILE_PATH = './projectdata/Age_limits.csv'
Public_use_CSV_FILE_PATH = './projectdata/Public_use.csv'
Liscence_retail_CSV_FILE_PATH = './projectdata/Licence_retail.csv'  # bool influence the retail sales first
Liscence_production_CSV_FILE_PATH = './projectdata/Licence_production.csv'
Tax_CSV_FILE_PATH = './projectdata/Tax.csv'   # bool
Community_actions_CSV_FILE_PATH = './projectdata/Community_action.csv'  #  bool
Health_warning_CSV_FILE_PATH = './projectdata/Health_warning.csv'
Consumption_CSV_FILE_PATH = './projectdata/Consumption.csv'
Countries_list_PATH = './dataprocessed/Country_list_final_2.csv'
Countries_location_PATH = './dataprocessed/Country_location.csv'
Countries_code_PATH = './dataprocessed/iso_countries_list.csv'

In [3]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

1. We have divided all the countries in the database into eight parts according to the United Nations geographical division criteria.
2. This is because we take into account that countries in the same region have similar political conditions, living habits and cultural background, such as countries in the EU, while countries in different regions are quite different in these aspects.
3. Therefore, our subsequent analysis will be based on the results of the following country segmentation.

In [4]:
def process_text(origin_text):
    new_text = origin_text.capitalize()
    new_text = ''.join(new_text.split('-'))
    new_text = ''.join(new_text.split(' '))
    new_text = ''.join(new_text.split('\t'))
    return new_text

In [5]:
def process_country_name(Filename, newFile_path, column):
    df = pd.read_csv(Filename, header=None, sep=',')
    for row in df.iterrows():
        country_name = row[1][column]
        if isinstance(country_name,str):
            row[1][column]=process_text(country_name)
    df.to_csv(newFile_path, header=None, index=False, sep=',')

In [6]:
def leave_out_nodata(Filename, newFile_path):
    df = pd.read_csv(Filename, header=None, sep=',')
    df = df[(df[2]!='No data')&(df[3]!='No data')&(df[4]!='No data')]
    df.to_csv(newFile_path,header=None,index=False,sep=',')

def leave_out_nodata_one(Filename, newFile_path):
    df = pd.read_csv(Filename, header=None, sep=',')
    df = df[df[2]!='No data']
    df.to_csv(newFile_path,header=None,index=False,sep=',')
    
def leave_out_nodata_multi(Filename, newFile_path):
    df = pd.read_csv(Filename, header=None, sep=',')
    df = df[(df[2]!='No data')&(df[3]!='No data')&(df[4]!='No data')&(df[5]!='No data')&(df[6]!='No data')&(df[7]!='No data')&(df[8]!='No data')&(df[9]!='No data')&(df[10]!='No data')]
    df.to_csv(newFile_path,header=None,index=False,sep=',')

In [7]:
# dict for file countriy_list_final, input country name and get iso3
def dict_country_code3(inputcountry):
    with open(Countries_list_PATH,'r',encoding="utf-8") as f:
        reader = csv.reader(f)
        fieldnames = next(reader) #get first row，as key for the dictionary get from generator next
        csv_reader = csv.DictReader(f,fieldnames=fieldnames) #self._fieldnames = fieldnames   # list of keys for the dict 以list的形式存放键名
        for row in csv_reader:
            d = {}
            for k,v in row.items():
                d[k]=v
            if d['Country']==inputcountry:
                outputcode = d['iso3']
                return(outputcode)
        return False

# dict for file countriy_list_final, input country name and get Region
def dict_country_region(inputcountry):
    with open(Countries_list_PATH,'r',encoding="utf-8") as f:
        reader = csv.reader(f)
        fieldnames = next(reader) #get first row，as key for the dictionary get from generator next
        csv_reader = csv.DictReader(f,fieldnames=fieldnames) #self._fieldnames = fieldnames   # list of keys for the dict 以list的形式存放键名
        for row in csv_reader:
            d = {}
            for k,v in row.items():
                d[k]=v
            if d['Country']==inputcountry:
                outputregion = d['Region']
                return(outputregion)
        return False

# dict for file countriy_list_final, input country name and get Religion
def dict_country_religion(inputcountry):
    with open(Countries_list_PATH,'r',encoding="utf-8") as f:
        reader = csv.reader(f)
        fieldnames = next(reader) #get first row，as key for the dictionary get from generator next
        csv_reader = csv.DictReader(f,fieldnames=fieldnames) #self._fieldnames = fieldnames   # list of keys for the dict 以list的形式存放键名
        for row in csv_reader:
            d = {}
            for k,v in row.items():
                d[k]=v
            if d['Country']==inputcountry:
                outputreligion = d['Religion']
                return(outputreligion)
        return False

In [8]:
# dict for file iso_countries_list, input iso3 and get iso2
def dict_country_code2(inputcode3):
    with open(Countries_code_PATH,'r',encoding="utf-8") as f:
        reader = csv.reader(f)
        fieldnames = next(reader)
        csv_reader = csv.DictReader(f,fieldnames=fieldnames) #self._fieldnames = fieldnames   # list of keys for the dict 以list的形式存放键名
        for row in csv_reader:
            d = {}
            for k,v in row.items():
                d[k]=v
            if d['iso3']==inputcode3:
                outputcode2 = d['iso2']
                return(outputcode2)
        return False

# dict for file countries_location_list, input iso2 and get latitude
def dict_country_latitude(inputcode2):
    with open(Countries_location_PATH,'r',encoding="utf-8") as f:
        reader = csv.reader(f)
        fieldnames = next(reader)
        csv_reader = csv.DictReader(f,fieldnames=fieldnames) #self._fieldnames = fieldnames   # list of keys for the dict 以list的形式存放键名
        for row in csv_reader:
            d = {}
            for k,v in row.items():
                d[k]=v
            if d['\ufeffiso2']==inputcode2:
                outputlatitude = d['latitude']
                return outputlatitude
        return False

# dict for file countries_location_list, input iso2 and get longitude
def dict_country_longitude(inputcode2):
    with open(Countries_location_PATH,'r',encoding="utf-8") as f:
        reader = csv.reader(f)
        fieldnames = next(reader)
        csv_reader = csv.DictReader(f,fieldnames=fieldnames) #self._fieldnames = fieldnames   # list of keys for the dict 以list的形式存放键名
        for row in csv_reader:
            d = {}
            for k,v in row.items():
                d[k]=v
            if d['\ufeffiso2']==inputcode2:
                outputlongitude = d['longitude']
                return outputlongitude
        return False

In [9]:
# dict for file Consumption , input countryname and get consumption
def dict_country_longitude(inputcode2):
    with open(Countries_location_PATH,'r',encoding="utf-8") as f:
        reader = csv.reader(f)
        fieldnames = next(reader)
        csv_reader = csv.DictReader(f,fieldnames=fieldnames) #self._fieldnames = fieldnames   # list of keys for the dict
        for row in csv_reader:
            d = {}
            for k,v in row.items():
                d[k]=v
            if d['\ufeffiso2']==inputcode2:
                outputlongitude = d['longitude']
                return outputlongitude
        return False

In [10]:
# test function dict_country print(dict_country('Switzerland'))

In [11]:
# # preprocess every file into country code
# def add_country_code(Filename, newFile_path):
#     df = pd.read_csv(Filename, header=None, sep=',')
#     df['CountryCode']='none'
#     # print(df.columns)
#     print(df.shape)
#     success_count = 0
#     for row in df.iterrows():
#         country_name = row[1][0]
#     #     print("Country name:", country_name)
#         outputcode = dict_country_code(country_name)
#         if(outputcode!=False):
#             row[1]['CountryCode']=outputcode
#             success_count = success_count + 1
#     print("Successfully Converted:", success_count)
#     # print(df.head())
#     df.to_csv(newFile_path,index=False,sep=',')

In [12]:
# country_to_code(Age_limits_CSV_FILE_PATH, "./dataprocessed/Age_limits_processed.csv")
# country_to_code(Public_use_CSV_FILE_PATH, "./dataprocessed/Public_use_processed.csv")

In [13]:
# preprocess add region and religion to each row
def add_code_region_religion(Filename, newFile_path):
    df = pd.read_csv(Filename, header=None, sep=',')
    df['CountryCode']='none'
    df['Region']='none'
    df['Religion']='none'
#     print(df.shape)
    success_count = 0
    for row in df.iterrows():
        country_name = row[1][0]
        country_name = process_text(country_name)
        outputcode = dict_country_code3(country_name)
        outputregion = dict_country_region(country_name)
        outputreligion = dict_country_religion(country_name)
        if outputregion!=False and outputreligion!=False:
            row[1][0]=country_name
            row[1]['CountryCode']=outputcode
            row[1]['Region']=outputregion
            row[1]['Religion']=outputreligion
            success_count = success_count + 1
    print("Successfully Converted:", success_count)
    df.to_csv(newFile_path, index=False, header=None, sep=',')

In [14]:
# preprocess countries location
def process_countries(Filename, newFile_path):
    df = pd.read_csv(Filename, header=None, sep=',', names=['Number', 'iso3', 'Country', 'Region', 'Religion'])
    df['iso2']='iso2'
    df['latitude']='latitude'
    df['longitude']='longitude'
#     print(df.shape)
    success_count = 0
    for row in df.iterrows():
        country_iso3 = row[1][1]
#         print("iso3", country_iso3)
        outputiso2 = dict_country_code2(country_iso3)
#         print("outputiso2", outputiso2)
        outputlatitude = dict_country_latitude(outputiso2)
        outputlongitude = dict_country_longitude(outputiso2)
        if outputlatitude!=False and outputlongitude!=False:
            row[1]['iso2']=outputiso2
            row[1]['latitude']=outputlatitude
            row[1]['longitude']=outputlongitude
            success_count = success_count + 1
    print("Successfully Converted:", success_count)
    df = df[['Number', 'Country', 'iso2', 'iso3', 'latitude', 'longitude', 'Region', 'Religion'] ]
    df.to_csv(newFile_path, index=False, header=None, sep=',')

In [15]:
def text_to_bool(Filename, new_file_path):
    df = pd.read_csv(Filename, header=None, sep=',')
    df.replace('Yes', '1', inplace=True)
    df.replace('No', '0', inplace=True)
    df.to_csv(new_file_path, header=None, index=False, sep=',')

def text_to_number(Filename, new_file_path):
    df = pd.read_csv(Filename, header=None, sep=',')
    df.replace('ban', '1', inplace=True)
    df.replace('partial restriction', '0.5', inplace=True)
    df.replace('voluntary/self-restricted', '0.3', inplace=True)
    df.replace('no restrictions', '0', inplace=True)
    df.to_csv(new_file_path, header=None, index=False, sep=',')

In [16]:
def preprocess_data_file(Filename, newFilename): # for Tax and Liscence
    tempFilename = './dataprocessed/tempFile.csv'
    text_to_bool(Filename, tempFilename)
    leave_out_nodata(tempFilename, newFilename)
    
def preprocess_data_file_one(Filename, newFilename): # for Tax and Liscence
    tempFilename = './dataprocessed/tempFile.csv'
    text_to_bool(Filename, tempFilename)
    leave_out_nodata_one(tempFilename, newFilename)

def preprocess_public_use_data(Filename, newFilename): # for Tax and Liscence
    tempFilename = './dataprocessed/tempFile.csv'
    text_to_number(Filename, tempFilename)
    leave_out_nodata_multi(tempFilename, newFilename)

In [17]:
# add_code_region_religion("./dataprocessed/Age_limits_processed.csv", "./dataprocessed/Age_limits_processed.csv")

## Process 4 simpler dataset: Tax, Liscence, Community_action, Health_warning 

1. Turning Yes/No to 1/0
2. Leave out "No data" rows

In [18]:
preprocess_data_file(Tax_CSV_FILE_PATH, "./dataprocessed/Tax_processed.csv")
preprocess_data_file(Liscence_production_CSV_FILE_PATH, "./dataprocessed/Liscence_production_processed.csv")
preprocess_data_file(Liscence_retail_CSV_FILE_PATH, "./dataprocessed/Liscence_retail_processed.csv")
preprocess_data_file(Community_actions_CSV_FILE_PATH, "./dataprocessed/Community_actions_processed.csv")
preprocess_data_file_one(Health_warning_CSV_FILE_PATH, "./dataprocessed/Health_warning_processed.csv")

FileNotFoundError: [Errno 2] File b'./projectdata/Tax.csv' does not exist: b'./projectdata/Tax.csv'

# Process public_use dataset, change text to valuse


ban                                               1

partial restriction                             0.5

voluntary/self-restricted                       0.3 #parameter that can be changed

no restrictions                                  0

No data                                          

In [ ]:
b = pd.read_csv(Public_use_CSV_FILE_PATH, header=None, sep=',')
b = pd.DataFrame(b)
for i in range(2, 4):
    print("Value_counts for column", i, "\n")
    print(b[i].value_counts())

In [ ]:
preprocess_public_use_data(Public_use_CSV_FILE_PATH, "./dataprocessed/Public_use_processed.csv")

# Process age_limit dataset, change text to valuse

ban                                               1

partial restriction                             0.5

voluntary/self-restricted                       0.3 #parameter that can be changed

no restrictions                                  0

No data       

In [ ]:
b = pd.read_csv(Age_limits_CSV_FILE_PATH, header=None, sep=',')
b = pd.DataFrame(b)
b[3].value_counts()

In [ ]:
add_code_region_religion(Age_limits_CSV_FILE_PATH, "./dataprocessed/Age_limits_processed.csv")

# 2. DataSets for Specific year 2016

## 2.1 DataSets Overview

**DataSet 1. Pure alcohol consumption, litres per capita, age 15+**

* Indicator full name: Pure alcohol consumption, litres per capita, age 15+

* Unit: litres per capita

* Dimensions:

* Country (COUNTRY) 

* Supranational group of countries (COUNTRY_GRP)

* Sex (SEX)

* Year of measure (YEAR)

* Years data is available: 1970—2015

In [ ]:
from IPython.core.display import display, HTML
display(HTML('<iframe frameborder="0" class="whoWidget" onload="(function (){if(!document.getElementById(\'ifr\')){var s=document.createElement(\'script\');s.type=\'text/javascript\';s.id=\'ifr\';s.src=\'https://gateway.euro.who.int/Scripts/libs/iframeResizer/iframeResizer.min.js\';document.getElementsByTagName(\'head\')[0].appendChild(s);}})();" src="https://gateway.euro.who.int/en/indicators/visualization-embed/?indId=18314&visId=19443" style="width: 100%;"></iframe>'))

**DataSet 2.Age limits - Alcohol service/sales by country**

* Data type: Count
* Topic: Risk factors
* Rationale: One first needs to be aware of a country's laws and policies regarding alcohol before one can establish whether these influence the level and patterns of alcohol consumption in the population. Thus, the legal status of alcohol production, distribution and consumption is an important indicator.
* Definition:The legal age limit for on-premise service of alcoholic beverages is defined as the age at which a person can be served alcoholic beverages on premises in a country, i.e. alcoholic beverages cannot be served to a person under this age. The legal age limit for off-premise sales of alcoholic beverages is defined as the age at which a person can be sold alcoholic beverages off premises in a country, i.e. alcoholic beverages cannot be sold to a person under this age.
* Disaggregation: Alcoholic beverage type
* Method of estimation: The national authorities of a given country respond to the Global Survey on Alcohol and Health.
* Expected frequency of data dissemination:Periodic
* Expected frequency of data collection:Periodic

Short name: Age limits on-/off-premise sales

Short name:Age limits on-/off-premise service

Total countries:194

In [ ]:
from IPython.core.display import display, HTML
display(HTML('<iframe id=\'rightframe\' name=\'leftiframe\' width="70%" src=\'http://apps.who.int/gho/data/node.main.A1144\'></iframe><br>'))

**DataSet 3.License requirement for alcohol production and retail sales**

* Short name: Licensing
* Data type: Categorical
* Topic: Risk factors
* Rationale: One first needs to be aware of a country's laws and policies regarding alcohol before one can establish whether these influence the level and patterns of alcohol consumption in the population. Thus, the legal status of alcohol production, distribution and consumption is an important indicator.
* Definition: Licensing is defined as the partial government control of production, and sale, wholesale and distribution of alcoholic beverages which requires licensing, as well as of import and export.
* Disaggregation: Alcoholic beverage type
* Method of estimation: The national authorities of a given country respond to the Global Survey on Alcohol and Health.
* Expected frequency of data dissemination:Periodic
* Expected frequency of data collection:Periodic

Retail sales by country

In [ ]:
display(HTML('<iframe id=\'rightframe\' name=\'leftiframe\' width="70%" src=\'http://apps.who.int/gho/data/node.main.A1179\'></iframe><br>'))

Production by country

In [ ]:
display(HTML('<iframe id=\'rightframe\' name=\'leftiframe\' width="70%" src=\'http://apps.who.int/gho/data/node.main.A1178\'></iframe><br>'))

## 2.1 Age-limit v.s. Pure alcohol consumption

In [ ]:
df = pd.read_csv("./dataprocessed/Age_limits_processed.csv", names=['Country', '2015-2017', '2009-2011', 'Age limits Beer'])
df.head(10)

## 2.2.Plot consumptions according to regions

### 2.2.1Merge dataset with Regions, Religions and Locations(Latitude, Longitude)

In [ ]:
df_country = pd.read_csv("./dataprocessed/Country_list_final.csv", header=None, sep=',')
df_country.head(10)

In [ ]:
df_latitude = pd.read_csv("./dataprocessed/Country_location.csv", header=None, sep=',')
df_latitude.head(10)

In [ ]:
df_country_code = pd.read_csv("./projectdata/deal_with_countries/iso_countries_list.csv", header=None, sep=',')
df_country_code.head(10)

In [ ]:
# Add latitude and longitude into country list
process_countries(Countries_list_PATH, './dataprocessed/countries_location_final.csv')
df_country_location = pd.read_csv('./dataprocessed/countries_location_final.csv', header=None, sep=',', names=['Number', 'Country', 'iso2', 'iso3', 'latitude', 'longitude', 'Region', 'Religion'])
df_country_location.head(10)

### 2.2.2 Merge Country list (Country, Region, Religion, Location) with Consumption

#### Preprocess consumption file's country name, Capitalized

In [ ]:
process_country_name(Consumption_CSV_FILE_PATH, './projectdata/Consumption_namecleaned.csv', 0)

#### Split consumption value and 95%CI 

In [ ]:
# Preprocess consumption file's
df_consumption = pd.read_csv("./projectdata/Consumption_namecleaned.csv", header=0, sep=',', names=['Country', '2015-2017', '2009-2011'])
df_consumption.head()
df_consumption['Alcohol_per_capita_2015-2017'], df_consumption['Alcohol_per_capita_2015-2017_95%CI'] = df_consumption['2015-2017'].str.split('[', 1).str
df_consumption['Alcohol_per_capita_2009-2011'], df_consumption['Alcohol_per_capita_2009-2011_95%CI'] = df_consumption['2009-2011'].str.split('[', 1).str
df_consumption = df_consumption[['Country', 'Alcohol_per_capita_2015-2017', 'Alcohol_per_capita_2015-2017_95%CI', 'Alcohol_per_capita_2009-2011', 'Alcohol_per_capita_2009-2011_95%CI']]
for row in df_consumption.iterrows():
    if isinstance(row[1][2], str):
        row[1][2] = row[1][2].replace("]", "")
    if isinstance(row[1][4], str):
        row[1][4] = row[1][4].replace("]", "")
df_consumption = df_consumption[1:]
df_consumption.to_csv("./projectdata/Consumption_final.csv", index=False, header=None, sep=',')
df_consumption.head(10)

In [ ]:
finaldf = pd.merge(df_country_location, df_consumption, on='Country')
finaldf.to_csv("./projectdata/Countries_finaldf.csv", index=False, sep=',', header=['Number', 'Country', 'iso2', 'iso_a3', 'latitude', 'longitude', 'Region', 'Religion', 'Alcohol_per_capita_2015-2017', 'Alcohol_per_capita_2015-2017_95%CI', 'Alcohol_per_capita_2009-2011', 'Alcohol_per_capita_2009-2011_95%CI'])
finaldf.head(10)

## Draw Maps

In [ ]:
%matplotlib inline
import matplotlib.pylab as pylab
pylab.rcParams['figure.figsize'] = 18, 18
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
fig, ax = plt.subplots(1, 1)
ax.set_title("Population by Country", fontsize=20)
world.plot(column='pop_est',
           ax=ax,
           legend=True,
           legend_kwds={'label': "Population by Country",'orientation': "horizontal"})
#            cmap='OrRd',
#            scheme='quantiles')

In [ ]:
world.head(5)

In [ ]:
'''Merging world shape data with our alchohol consumption data'''
finaldf = finaldf.rename(columns={'iso3':'iso_a3'})
world = world.merge(finaldf, on='iso_a3')
world.head(5)

In [ ]:
world.columns.values.tolist()

In [ ]:
world = world.rename(columns={'Alcohol_per_capita_2015-2017':'Alcohol_com1'})
world = world.rename(columns={'Alcohol_per_capita_2009-2011':'Alcohol_com2'})
world['Alcohol_com1'] = world.Alcohol_com1
world['Alcohol_com2'] = world.Alcohol_com2
world['iso_a3'] = world.iso_a3
world['Region'] = world.Region
world['Religion'] = world.Religion
world.columns.values.tolist()
world['iso_a3'].head(10)
world['Alcohol_com1'].max()
world['Alcohol_com1'].min()

In [ ]:
import matplotlib.pylab as pylab
pylab.rcParams['figure.figsize'] = 18, 18
fig, ax = plt.subplots(1, 1)
ax.set_title("Avg Alcohol consumption 2015-2017 by Country", fontsize=20)
world.plot(column='Alcohol_com1',
           ax=ax,
           legend=True,
#            legend_kwds={'labels': "Avg Alcohol_2015-2017 by Country",'legend': "horizontal"})
           cmap='OrRd',
           scheme='quantiles')

In [ ]:
import matplotlib.pylab as pylab
pylab.rcParams['figure.figsize'] = 18, 18
fig, ax = plt.subplots(1, 1)
ax.set_title("Avg Alcohol consumption 2009-2011 by Country", fontsize=20)
world.plot(column='Alcohol_com2',
           ax=ax,
           legend=True,
#            legend_kwds={'labels': "Avg Alcohol_2015-2017 by Country",'legend': "horizontal"})
           cmap='OrRd',
           scheme='quantiles')

Get those countries that experienced dramatically Alcohol consumption changes between two time periods

In [ ]:
from interval import Interval
zoom_1 = Interval(0.0, 8.0, higher_closed=False)
zoom_2 = Interval(8.0, 23.0, higher_closed=False)
zoom_3 = Interval(23.0, 45.0, higher_closed=False)
zoom_4 = Interval(45.0, 66.0, higher_closed=False)
zoom_5 = Interval(66.0, 86.0)
intervals = [zoom_1, zoom_2, zoom_3, zoom_4, zoom_5]

In [ ]:
dramatically_countries=[]
sum_con=0
for row in world.iterrows():
    sum_con+=1
    country_iso_3 = row[1][3]
#     print(country_iso_3)
    comp1 = float(row[1][13])
#     print(comp1)
    comp2 = float(row[1][15])
#     print(comp2)
    interval1 = intervals[0]
    interval2 = intervals[0]
    for i in intervals:
        if comp1 in i:
            interval1 = i
        if comp2 in i:
            interval2 = i
#     print(interval1)
#     print(interval2)
    if interval1 != interval2:
        print(comp1, comp2, comp1-comp2)
        dramatically_countries.append(country_iso_3)
print(sum_con)
print("countries that alcohol consumption changed dramatically", dramatically_countries)
print(len(dramatically_countries))

In [ ]:
world['change'] = world.apply(lambda x: float(x.Alcohol_com1) - float(x.Alcohol_com2), axis=1)

In [ ]:
world.head(10)

In [ ]:
world.loc[world['change']>0.5]

In [ ]:
world['change_label'] = world.apply(lambda x: 1 if float(x.change)>0.5 else 0, axis=1)

In [ ]:
import matplotlib.pylab as pylab
pylab.rcParams['figure.figsize'] = 18, 18
fig, ax = plt.subplots(1, 1)
ax.set_title("Countries that Avg Alcohol consumption changed dramatically during two periods", fontsize=20)
world.plot(column='change_label',
           ax=ax,
           legend=True,
#            legend_kwds={'labels': "Avg Alcohol_2015-2017 by Country",'legend': "horizontal"})
           cmap='OrRd',
           scheme='quantiles')

In [ ]:
world_changed.describe()

Plot Regions by country

In [ ]:
import matplotlib.pylab as pylab
pylab.rcParams['figure.figsize'] = 18, 18
fig, ax = plt.subplots(1, 1)
ax.set_title("Region", fontsize=20)
world.plot(column='Region',
           ax=ax,
           legend=True)

In [ ]:
world['Religion'].unique()

In [ ]:
for row in world.iterrows():
    country_religion = row[1][12]
#     print(country_religion)
    if isinstance(country_religion,str):
        row[1][12] = process_text(country_religion)
world['Religion'].unique()

In [ ]:
import matplotlib.pylab as pylab
pylab.rcParams['figure.figsize'] = 18, 18
fig, ax = plt.subplots(1, 1)
ax.set_title("Religion", fontsize=20)
world.plot(column='Religion',
           ax=ax,
           legend=True)

In [ ]:
import folium

# define the world map
world_map = folium.Map()

# display world map
world_map

In [ ]:
# # San Francisco latitude and longitude values
# latitude = 37.77
# longitude = -122.42

# # Create map and display it
# san_map = folium.Map(location=[latitude, longitude], zoom_start=12)

# # Display the map of San Francisco
# san_map

In [ ]:
# Read Dataset 
cdata = pd.read_csv("./projectdata/Countries_finaldf.csv", header=None, sep=',', names=['Number', 'Country', 'iso2', 'iso3', 'latitude', 'longitude', 'Region', 'Religion', 'Alcohol_per_capita_1', 'Alcohol_per_capita_1_ci', 'Alcohol_per_capita_2', 'Alcohol_per_capita_2_ci'])
# cdata['latitude'].astype('float')
cdata.info()
cdata.head(10)

In [ ]:
def location_to_float(value):
#     print(value)
    if value!='latitude' and value!='longitude':
        return np.float(value)
    else:
        return np.nan
cdata['latitude'] = cdata['latitude'].apply(location_to_float)
cdata['longitude'] = cdata['longitude'].apply(location_to_float)
# cdata['Alcohol_per_capita_1'] = cdata['Alcohol_per_capita_1'].apply(location_to_float)
cdata.info()
print(cdata.shape)
cdata.head()

In [ ]:
cdata = cdata.dropna(axis=0, how='any')
cdata.to_csv("./projectdata/Countries_dropped.csv", header=None, index=False, sep=',')
# cdata.isnull().all(axis=0)
cdata.shape

In [ ]:
# cdata['latitude'].value_counts()
print(np.any(cdata.isnull())==True)

In [ ]:
# Instantiate a feature group for the incidents in the dataframe
incidents = folium.map.FeatureGroup()
# Loop through the consumption data and add each to the incidents feature group

for lat, lng, value in zip(cdata.latitude, cdata.longitude, cdata.Alcohol_per_capita_1):
    incidents.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=7, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.4
        )
    )

# Visualize countries' location

In [ ]:
# Add incidents to map
# world_map = folium.Map(location=[latitude, longitude], zoom_start=12)
world_map = folium.Map(zoom_start=12)
world_map.add_child(incidents)

# Visualize Regional countries

In [ ]:
from folium import plugins

# let's start again with a clean copy of the map of San Francisco
world_map = folium.Map(zoom_start = 12)

# # instantiate a mark cluster object for the incidents in the dataframe
# incidents = plugins.MarkerCluster().add_to(world_map)

# loop through the dataframe and add each data point to the mark cluster
for lat, lng, consumption in zip(cdata.latitude, cdata.longitude, cdata.Alcohol_per_capita_1):
    folium.Marker(
        location=[lat, lng],
        popup=consumption,
        icon=None,
        value=consumption,
    ).add_to(incidents)

# add incidents to map
world_map.add_child(incidents)

# Plot Age_limits with consumption

In [ ]:
df_test = df[2:12]
plt.bar(df_test['Country'], df_test['2015-2017']) # plot
plt.xticks(rotation=30)

In [ ]:
total_width, n = 0.8, 2
width = total_width / n
 
plt.bar(df_test['Country'], df_test['2015-2017'], width=width, label='consumption',fc = 'y')
for item in df_test['Country']:
    item = item + width
plt.bar(x, df_test['Age limits Beer'], width=width, label='Age_limits',tick_label = df_test['Country'],fc = 'r')
plt.xticks(rotation=30)
plt.legend()
plt.show()